In [8]:
import nbimporter
import torch
import torch.nn as nn
import torch.functional as f

import numpy as np
import matplotlib.pyplot as plt

from tqdm import trange
from IPython.display import clear_output
from torch.optim import Adam
from pogema_env import My_env

In [2]:
kernel_size = 3
padding = 1
in_ch = 3
hidden_sizes = [128, 128, 128, 256, 256, 128]
n_actions = 5

In [3]:
class Primal_network(nn.Module):
    def __init__(self, n_actions=n_actions, epsilon=0):
        super().__init__()
        self.epsilon = epsilon
        self.n_action = n_actions
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=hidden_sizes[0], kernel_size=kernel_size, padding=1),
            nn.Conv2d(in_channels=hidden_sizes[0], out_channels=hidden_sizes[1], kernel_size=kernel_size, padding=1),
            nn.Conv2d(in_channels=hidden_sizes[1], out_channels=hidden_sizes[2], kernel_size=kernel_size, padding=1),
        )
        
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        
        self.next_layers = nn.Sequential(
            nn.Conv2d(in_channels=hidden_sizes[2], out_channels=hidden_sizes[3], kernel_size=kernel_size, padding=1),
            nn.Conv2d(in_channels=hidden_sizes[3], out_channels=hidden_sizes[4], kernel_size=kernel_size, padding=1),
            nn.Conv2d(in_channels=hidden_sizes[4], out_channels=hidden_sizes[5], kernel_size=kernel_size, padding=1),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=hidden_sizes[5], out_channels=512, kernel_size=2)
        )
        
        self.linear = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512)
        )
        
        self.lstm = nn.LSTM(512 * 2, 512)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(512, n_actions + 2)
        
    def forward(self, input_):
        print("input.shape = ", input_.shape)
        out_res = self.conv_layers(input_)
        print("First_conv = ", out_res.shape)
        out_res = self.max_pool(out_res)
        print("Max_pool = ", out_res.shape)
        out_res = self.next_layers(out_res)
        out_res.squeeze_()
        print("Second conv = ", out_res.shape)
        out = self.linear(out_res)
        print("After_linear = ", out.shape)
        
        concat = torch.concatenate((out, out_res), dim=-1)
        print("concatination = ", concat.shape)
        lstm_input = self.relu(concat)
        print(lstm_input.shape)
        out, (_ , _) = self.lstm(lstm_input)
        print("LSTM = ", out.shape)
        return self.fc(out)

In [4]:
pr = Primal_network()

In [5]:
inp = torch.ones(5, 3, 11, 11)
inp.shape, hidden_sizes[0], in_ch

(torch.Size([5, 3, 11, 11]), 128, 3)

In [6]:
pr(inp)

input.shape =  torch.Size([5, 3, 11, 11])
First_conv =  torch.Size([5, 128, 11, 11])
Max_pool =  torch.Size([5, 128, 5, 5])
Second conv =  torch.Size([5, 512])
After_linear =  torch.Size([5, 512])
concatination =  torch.Size([5, 1024])
torch.Size([5, 1024])
LSTM =  torch.Size([5, 512])


tensor([[ 0.0135,  0.0152, -0.0317,  0.0415, -0.0085, -0.0490,  0.0177],
        [ 0.0112,  0.0171, -0.0333,  0.0472, -0.0092, -0.0573,  0.0231],
        [ 0.0104,  0.0181, -0.0355,  0.0508, -0.0086, -0.0617,  0.0260],
        [ 0.0102,  0.0187, -0.0374,  0.0532, -0.0076, -0.0641,  0.0275],
        [ 0.0102,  0.0190, -0.0387,  0.0548, -0.0068, -0.0652,  0.0283]],
       grad_fn=<AddmmBackward0>)

In [9]:
def backward_reward(reward, is_done, gamma=0.99):
    ln, time = rewards.shape[:-2]
    result = rewards[:, -1]
    
    for t in range(time - 2, -1, -1):
        result[:, t] = reward[:, t] + gamma * (1 - is_done[:, t]) * result[:, t+1] 
    
    return result

In [ ]:
def compute_loss(s, action, reward, next_s, is_done):
    s = torch.Tensor(s)
    action = torch.Tensor(action)
    next_s = torch.Tensor(next_s)
    reward = torch.Tensor(reward)
    is_done = torch.Tensor(is_done)
    
    reward = 

In [7]:
primal = Primal_network(policy, optimizer)
optimizer =  Adam(pr.parameters(), lr=7e-4, eps=1e-5)

state = My_env.reset()

step = 0
total_steps = 10 ** 7

with trange(step, total_steps + 1, 40) as progress_bar:
    for step in progress_bar:
        # play
        trajectory = runner.get_next()


        NumpySummaries.set_step(runner.step_var)
        # train
        if step % 50 == 0:
            policy_loss, entropy, advatages = a2c.policy_loss(trajectory)
            value_loss = a2c.value_loss(trajectory)
            loss_ = a2c.loss(trajectory)
            
            env.add_summary_scalar('policy_loss', policy_loss.item())
            env.add_summary_scalar('value_loss', value_loss.item())
            env.add_summary_scalar('loss', loss_.item())
            env.add_summary_scalar('entropy', entropy.item())
            env.add_summary_scalar('advatages', advatages.item())

            mean_reward = NumpySummaries.get_values("SpaceInvadersNoFrameskip-v4/reward_mean_100")
            policy_loss_history = NumpySummaries.get_values("policy_loss")
            value_loss_history = NumpySummaries.get_values("value_loss")
            loss_history = NumpySummaries.get_values("loss")
            entropy_history = NumpySummaries.get_values("entropy")
            advatages_history = NumpySummaries.get_values("advatages")

        a2c.step(trajectory)
        scheduler.step()

        with torch.no_grad():
            if step % 1000 == 0:
                clear_output()
                plt.figure(figsize=[16, 9])

                plt.subplot(2, 3, 1)
                plt.title("Mean reward per life")
                plt.plot([m[1] for m in mean_reward])
                plt.grid()

                plt.subplot(2, 3, 2)
                plt.title("loss_history")
                plt.plot([m[1] for m in loss_history])
                plt.grid()
                
                plt.subplot(2, 3, 3)
                plt.title("policy_loss_history")
                plt.plot([m[1] for m in policy_loss_history])

                plt.grid()

                plt.subplot(2, 3, 4)
                plt.title("value_loss_history")
                plt.plot([m[1] for m in value_loss_history])
                plt.grid()

                plt.subplot(2, 3, 5)
                plt.title("entropy_history")
                plt.plot([m[1] for m in entropy_history])
                plt.grid()

                plt.subplot(2, 3, 6)
                plt.title("advatages_history")
                plt.plot([m[1] for m in advatages_history])
                plt.grid()
                

                plt.show()

NameError: name 'policy' is not defined